In [1]:
import pandas as pd
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from tqdm import tqdm_notebook as tqdm
from tqdm import trange

In [22]:
df = pd.read_csv('dataframe/df_completec.csv')[['id','text1']]
df_liwc = pd.read_csv('dict/df_liwc.csv')
dizionario = open("dict/liwc_dic.json")

In [16]:
df_liwc = df_liwc.fillna(0)
df_liwc

,Unnamed: 0,a,abandon*,abdomen*,abilit*,able*,abortion*,about,above,abrupt*,...,yum*,yup,zen,zero,zillion*,zion*,zit,zits,zoloft,zz*
0,0,funct,affect,bio,achieve,achieve,bio,funct,funct,time,...,percept,assent,relig,funct,funct,relig,bio,bio,bio,nonfl
1,1,article,negemo,body,0,0,health,adverb,preps,relativ,...,0,0,0,number,number,0,health,health,health,0
2,2,0,sad,0,0,0,sexual,preps,space,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,cogmech,0,0,0,0,0,relativ,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,inhib,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
type(dizionario)

_io.TextIOWrapper

In [20]:
k = list(dizionario.keys())
sentiment_key = list(dizionario.keys())

AttributeError: '_io.TextIOWrapper' object has no attribute 'keys'

In [17]:
#Create a new dataframe df2
df2 = pd.DataFrame(
                  index=pd.Index(sentiment_key),
                  columns=pd.Index(label)).fillna(0)
df2.head()

,0,1,2,3,4,5,6,7
Unnamed: 0,0,1,2,3,4,5,6,7
a,funct,article,0,0,0,0,0,0
abandon*,affect,negemo,sad,cogmech,inhib,0,0,0
abdomen*,bio,body,0,0,0,0,0,0
abilit*,achieve,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
zion*,relig,0,0,0,0,0,0,0
zit,bio,health,0,0,0,0,0,0
zits,bio,health,0,0,0,0,0,0
zoloft,bio,health,0,0,0,0,0,0


In [ ]:
def new_df(sent,df,df2):
    df2[sent] = df.apply(lambda i: i.astype(str).str.contains(sent).any(), axis=0).astype(int)
    return df2

In [15]:
df_liwc[['abs','abuse*']]

,abs,abuse*
0,bio,affect
1,body,negemo
2,NaN,anger
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN


In [ ]:
def text_emotion(df, column):
    '''
    INPUT: DataFrame, string
    OUTPUT: the original DataFrame with ten new columns for each emotion
    '''

    new_df = df.copy()

    xlsx = pd.read_excel(r'C:\Users\Nick\Desktop\GitProjects\NLP_projects\The_Silmarillion\NRC-Sentiment-Emotion-Lexicons\NRC-Sentiment-Emotion-Lexicons\NRC-Emotion-Lexicon-v0.92\NRC-Emotion-Lexicon-v0.92-In105Languages-Nov2017Translations.xlsx')
    emolex_df = xlsx[['word', 'Positive_NRC','Negative_NRC','Anger', 'Anticipation', 'Disgust', 'Fear','Joy',
                      'Sadness', 'Surprise', 'Trust']]
    emotions = emolex_df.columns.drop('word')
    emo_df = pd.DataFrame(0, index=df.index, columns=emotions)

    stemmer = SnowballStemmer("english")

    
    with tqdm(total=len(list(new_df.iterrows()))) as pbar:
        for i, row in new_df.iterrows():
            pbar.update(1)
            document = word_tokenize(new_df.loc[i][column])
            for word in document:
                word = stemmer.stem(word.lower())
                emo_score = emolex_df[emolex_df.word == word]
                if not emo_score.empty:
                    for emotion in list(emotions):
                        emo_df.at[i, emotion] += emo_score[emotion]

    new_df = pd.concat([new_df, emo_df], axis=1)

    return new_df